
# N2 Configuration with varying outer air region thickness and Wavelength

Here we create (hopefully) an interactive plot to see what happens to the modes as we vary the outer air region thickness.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from fiberamp.fiber.microstruct.bragg import BraggExact
from step_exact import plotlogf, plotlogf_real
from ngsolve.webgui import Draw
from scipy.optimize import newton

from ipywidgets import interactive, FloatSlider

importing NGSolve-6.2.2203-56-g47f483033


In [2]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, 5/3 * d, 1.5e-5]
outer = 'h2'
nu = 1

# Both Wavelength and T

In [3]:
d = 4.0775e-05  # thickness of innermost region (core radius)
ts = [d, 1e-5, d, 1.5e-5]
no_mesh = True
H = BraggExact(ts=ts, maxhs=[.1, .03, .06, .1], no_mesh=no_mesh)


In [4]:
nu = 1

def det_plot(T, wl):
    d = 4.0775e-05  # thickness of innermost region (core radius)
    ts = [d, 1e-5, T * d, 1.5e-5]
    H.ts = ts
    H.wavelength = wl * 1e-6
    cutoff = H.ks[0] * H.scale
    
    Xr = np.linspace(.9998* cutoff , 1.000001*cutoff , num=75)
    Xi = np.linspace( -.001, .001, num=75)
    xr, xi = np.meshgrid(Xr, Xi)
    zs = xr + 1j * xi
    
    fig, ax1 = plt.subplots(1, 1, figsize=(12,8))
    
    fs1 = H.determinant(zs, nu=nu, outer='h2')
    ax1.contour(xr, xi, np.log(np.abs(fs1)), levels=75)
    ax1.grid(True)
    m, M = ax1.get_ylim()
    ax1.plot([cutoff, cutoff], [m,M],lw=4, c="r")
    ax1.set_facecolor('grey')
    ax1.set_title('Behavior near Fundamental Mode as outer thickness changes.')
#     ax1.set_xticklabels([])
    ax1.set_yticks([0])
    plt.show()
    
interactive_plot = interactive(det_plot,
                               T=FloatSlider(min=1, max=15, step=.05, value=1.83, readout_format='.2f'),
                               wl=FloatSlider(min=1.4, max=2, step=.0005, value=1.602, readout_format='.2f'))

output = interactive_plot.children[-1]
output.layout.height = '10'
interactive_plot

interactive(children=(FloatSlider(value=1.83, description='T', max=15.0, min=1.0, step=0.05), FloatSlider(valu…

In [5]:
A = BraggExact(ts=H.ts)


In [6]:
Draw(A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

# The Bumping is much more dramatic at some wavelengths.

At 1.89, the fundamental runs from the incoming modes right away as T increases from 1.

Would be worth looking at mode profiles for this.

In [48]:
k_low = H.ks[0] * H.scale
guess = .99985*k_low

In [49]:
from scipy.optimize import newton
guess = np.array(guess)

beta1 = newton(H.determinant, guess, args=(nu, outer), tol = 1e-15)

print("Scaled beta: ", beta1, ". Residual of determinant: ", abs(H.determinant(beta1, nu, outer)))


Scaled beta:  (49.8587070037973-7.394985868274408e-05j) . Residual of determinant:  2.4501715255946905e-13


In [50]:
Fs = H.all_fields(beta1, nu, outer)

In [51]:
Draw(Fs['Etv'].real, A.mesh)

WebGuiWidget(value={'ngsolve_version': '6.2.2203-56-g47f483033', 'mesh_dim': 2, 'order2d': 2, 'order3d': 2, 'd…

BaseWebGuiScene

# Same mode profile pattern holds

The bumped mode goes from fundamental to large area, middle mode becomes new fundamental, left mode is sort of blend.